In [ ]:
import torch
import numpy as np
import torchvision.datasets as datasets

In [ ]:
print("torch version: \t", torch.__version__)
print("np version: \t", np.__version__)

In [ ]:
root = "../data/"
dataset_cat = "medium"

val_cls = root + "validation_classification/" + dataset_cat
test_cls = root + "test_classification/" + "medium"
# train_cls = root + "train_data/" + dataset_cat # for actual training
train_cls = val_cls # for development

val_vrf = root + "validation_verification"
test_vrf = root + "test_verification"

In [ ]:
train_dataset = datasets.ImageFolder(root = train_cls, 
                     transform=None, 
                     target_transform=None, 
                     loader=None, 
                     is_valid_file=None)